In [1]:
from langchain_community.document_loaders import PyPDFLoader
from pathlib import Path
import re
import os
from api import GOOGLE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [2]:
papers_path = Path("papers/selection")
def load_all_files(root_path, mode="page"):
    docs = []
    for path, folders, files in root_path.walk():
        if len(folders) != 0:
            continue
        for file in files:
            if not file.endswith(".pdf"):
                continue
            doc_loader = PyPDFLoader(
                path / file,
                mode=mode,
                extraction_mode="layout"
            )
            pages = doc_loader.load()
            docs.extend(pages)
        
    return docs

docs = load_all_files(papers_path)
        

Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text

In [3]:
def preprocess_documents(docs):
    for doc in docs:
        doc.page_content = re.sub(" +", " ", doc.page_content)
    
    return docs
docs = preprocess_documents(docs)

In [5]:
from langchain_core.prompts import PromptTemplate

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use around three sentences for your answer and keep it concise.
Do not ever mention that you are using the context as source.

Question: {question}

Context:
{context}

Answer:"    
"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [6]:
# from langchain import hub
# from langchain_chroma import Chroma
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from qdrant_client import models
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
client = QdrantClient(path="papers_db")
client.create_collection(
    collection_name="selection",
    vectors_config=models.VectorParams(
        size=768,
        distance=models.Distance.COSINE
    )

)
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
# vectorstore = Chroma.from_documents(
#     documents=splits,
#     embedding=embeddings,
#     persist_directory="selection_db"
#  )
# vectorstore = Chroma(
#     persist_directory="selection_db",
#     embedding_function=embeddings
# )
vectorstore = QdrantVectorStore(
    client=client,
    collection_name="selection",
    embedding=embeddings,
    distance=models.Distance.COSINE
)
vectorstore.add_documents(splits)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
prompt = custom_rag_prompt


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

# rag_chain.invoke("What is Task Decomposition?")
system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer the question."
    "If you don't know the answer, just say that you don't know."
    "Use around three sentences for your answer and keep it concise."
    "Do not ever mention that you are using the context as source."
    "Here is context for your answer:"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

rag_chain.invoke({"input": "What kind of material is MAPbI3?"})

{'input': 'What kind of material is MAPbI3?',
 'context': [Document(metadata={'producer': 'PDFlib+PDI 9.1.2p4 (C++/Win64); modified using iText 4.2.0 by 1T3XT', 'creator': 'Arbortext Publishing Engine', 'creationdate': '2024-12-31T14:36:36-05:00', 'moddate': '2025-03-26T08:25:41-07:00', 'fpage': '184', 'acs openaccess version': 'v3.2', 'subject': 'J. Phys. Chem. Lett. 2025.16:184-190', 'printengine': 'Arbortext Advanced Print Publisher 11.2.5208/W Library-x64', 'author': 'Pelayo Marin-Villa, Mattia Gaboardi, Boby Joseph, Frederico Alabarse, Jeff Armstrong, Kacper Drużbicki, and Felix Fernandez-Alonso', 'title': 'Methylammonium Lead Iodide across Physical Space: Phase Boundaries and Structural Collapse', 'epsprocessor': 'PStill version 1.84.42', 'prism:doi': '10.1021/acs.jpclett.4c03336', 'lpage': '190', 'prism:versionidentifier': 'Issue', 'jcalibrarybuild': '5.2.i3', 'acs openaccess stamp': '01/02/2025 15:17:17 (CCBY4_0)', 'jav:journal_article_version': 'VoR', 'source': 'papers\\select

In [9]:
rag_chain.invoke({"input": "What were the technical specs of the quantum computer used by Google in its 'quantum supremacy' experiment?"})

{'input': "What were the technical specs of the quantum computer used by Google in its 'quantum supremacy' experiment?",
 'context': [Document(metadata={'producer': 'PyPDF', 'creator': 'Springer', 'creationdate': '2019-10-17T13:39:15+05:30', 'author': 'Frank Arute', 'crossmarkdomains[1]': 'springer.com', 'crossmarkdomains[2]': 'springerlink.com', 'keywords': '', 'moddate': '2019-10-17T13:40:07+05:30', 'subject': 'Nature, doi:10.1038/s41586-019-1666-5', 'title': 'Quantum supremacy using a programmable superconducting processor', 'doi': '10.1038/s41586-019-1666-5', 'source': 'papers\\selection\\4_quantum_supremacy.pdf', 'total_pages': 7, 'page': 1, 'page_label': '506', '_id': '4b043942afa442fd8d4bfeca7ca8eb8d', '_collection_name': 'selection'}, page_content='the quantum states we intend to create are sensitive to errors. A single Each qubit is connected to a linear resonator used to read out the qubit\nbit or phase flip over the course of the algorithm will completely shuffle state5. As 

In [6]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")

In [7]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, SystemMessage
import base64

# Function to encode an image as base64
def encode_image(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode("utf-8")

# Encode the image(s)
image_path = "image.jpg"  # Replace with your image path
image_base64 = encode_image(image_path)

# Define system message (instructions for the AI)
system_template = "You are an AI assistant that analyzes images and answers user queries."
system_message = SystemMessagePromptTemplate.from_template(system_template)

# Define human message (text + image context)
human_template = "{text}"  # Placeholder for user input text
human_message = HumanMessagePromptTemplate.from_template(human_template)

# Create the chat prompt template
chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

# Format the chat prompt with user input
formatted_prompt = chat_prompt.format_messages(
    text="What can you tell me about this image?",
)

# Append image to the human message dynamically
formatted_prompt.append(
    HumanMessage(
        content=[
            {"type": "text", "text": "Here is the image context:"},
            {"type": "image_url", "image_url": f"data:image/jpeg;base64,{image_base64}"}
        ]
    )
)

# Initialize Gemini model (multimodal)
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# Get the response from the model
response = llm(formatted_prompt)
print(response.content)

/var/folders/9w/3l6rxjcs3bgfl3v7b2f66llr0000gn/T/ipykernel_77878/2457562682.py:44: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(formatted_prompt)


The image features a tabby cat wearing orange sunglasses. The cat has green eyes and a white chest. It's sitting on what appears to be a sofa, with various cushions visible in the background. The image is well-lit and has a shallow depth of field, focusing attention on the cat's face.


In [ ]:
from langchain_core.runnables import RunnableLambda

top_k = 2

def retrieve_images(query):
    return [encode_image("image.jpg") for i in range(top_k)]

def print_thing(thing):
    print(thing)
    return thing

def expand_context_images(message):
    new_message = {}
    new_message["query"] = message["query"]
    for i, image in enumerate(message["context_images"]):
        new_message[f"context{i}"] = image
    
    return new_message

system_message = (
    "You are an assistant for question-answering tasks."
    "Use the images of retrieved context to answer the question."
    "If you don't know the answer, just say that you don't know."
    "Use around three sentences for your answer and keep it concise."
    "Do not ever mention that you are using the context as source."
) 

user_message = [
    {
        "type": "text",
        "text": "{query}",
    }
] + [
    {
        "type": "image_url",
        "image_url": {"url": "data:image/jpeg;base64,{placeholder}".replace("placeholder", f"context{i}")},
    }
    for i in range(top_k)
]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        (
            "user",
            user_message,
        ),
    ]
)
# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_message),
#         (
#             "user",
#             [
#                 {
#                     "type": "text",
#                     "text": "{query}",

#                 },
#                 {
#                     "type": "image_url",
#                     "image_url": {"url": "data:image/jpeg;base64,"},
#                 }
#             ],
#         ),
#     ]
# )

rag_chain = (
    {"context_images": retrieve_images, "query": RunnablePassthrough()}
    | RunnableLambda(expand_context_images)
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What are these images? Are they the same?")

'The images show a cat wearing orange sunglasses. Yes, they are the same.'

In [42]:
prompt

ChatPromptTemplate(input_variables=['context0', 'context1', 'query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.Use the images of retrieved context to answer the question.If you don't know the answer, just say that you don't know.Use around three sentences for your answer and keep it concise.Do not ever mention that you are using the context as source."), additional_kwargs={}), HumanMessagePromptTemplate(prompt=[PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), ImagePromptTemplate(input_variables=['context0'], input_types={}, partial_variables={}, template={'url': 'data:image/jpeg;base64,{context0}'}), ImagePromptTemplate(input_variables=['context1'], input_types={}, partial_variables={}, template={'url': 'data:image/jpeg;base64,{context1}'})], additional_kwargs={}